In [116]:
import argparse
import os
import sys
import pickle
import sklearn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37184 entries, 0 to 37183
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   real review?  37184 non-null  int64 
 1   category      37184 non-null  object
 2   rating        37184 non-null  int64 
 3   text_         37184 non-null  object
 4   Unnamed: 4    86 non-null     object
 5   Unnamed: 5    11 non-null     object
 6   Unnamed: 6    5 non-null      object
 7   Unnamed: 7    2 non-null      object
 8   Unnamed: 8    2 non-null      object
 9   Unnamed: 9    2 non-null      object
 10  Unnamed: 10   2 non-null      object
 11  Unnamed: 11   2 non-null      object
 12  Unnamed: 12   2 non-null      object
 13  Unnamed: 13   2 non-null      object
 14  Unnamed: 14   2 non-null      object
 15  Unnamed: 15   2 non-null      object
 16  Unnamed: 16   2 non-null      object
 17  Unnamed: 17   1 non-null      object
 18  Unnamed: 18   1 non-null      object
dtypes: i

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [110]:
file_train = pd.read_csv("Desktop/fake-reviews/reviews_train.csv", 
                         usecols=[0,1,2,3],
                         dtype={'real review?': float,
                                'category': str, 
                                #'rating': int, 
                                'text_': str})

vectorizer = TfidfVectorizer(max_features=5000)
# Transform text data to list of strings
corpora = file_train['text_'].astype(str).values.tolist()
# Obtain featurizer from data
vectorizer.fit(corpora)
# Create feature vector
X = vectorizer.transform(corpora)
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
rating = pd.get_dummies(file_train['rating'])
category = pd.get_dummies(file_train['category'])
df = pd.DataFrame(category)
df = df.join(rating)
X = X.join(df)

In [118]:
file_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37184 entries, 0 to 37183
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   real review?  37184 non-null  float64
 1   category      37184 non-null  object 
 2   rating        37184 non-null  int64  
 3   text_         37184 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [111]:
lr = SVC(kernel='linear')
# # You can safely ignore any "ConvergenceWarning" warnings

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_sc = sc.fit_transform(X)
lr.fit(X, file_train['real review?'])

lr.score(X, file_train['real review?'])
# Get logistic regression predictions
#y_pred = svm.predict_proba(X)

# Accuracy of predictions with the true labels and take the percentage
# Because our dataset is balanced, measuring just the accuracy is OK
#accuracy = (y_pred == file_train['real review?']).sum() / file_train['real review?'].size

0.9328474612736661

In [112]:
file_validation = pd.read_csv("Desktop/fake-reviews/reviews_validation.csv", 
                         usecols=[0,1,2,3],
                         dtype={'real review?': float,
                                'category': str, 
                                #'rating': int, 
                                'text_': str})

corpora = file_validation['text_'].astype(str).values.tolist()
# Use the words obtained in training to encode in testing
X = vectorizer.transform(corpora)
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
rating = pd.get_dummies(file_validation['rating'])
category = pd.get_dummies(file_validation['category'])
df = pd.DataFrame(category)
df = df.join(rating)
X = X.join(df)

lr.score(X, file_validation['real review?'])
# Get logistic regression predictions
#y_pred = svm.predict_proba(X)

# Accuracy of predictions with the true labels and take the percentage
# Because our dataset is balanced, measuring just the accuracy is OK
#accuracy = (y_pred == file_train['real review?']).sum() / file_train['real review?'].size


0.9069069069069069

In [80]:
#print(y_pred)

In [113]:
file_test = pd.read_csv("Desktop/fake-reviews/reviews_test_attributes.csv",quotechar='"',usecols=[0,1,2,3],dtype={'real review?': float,'category': str,'text_': str})
#'rating': int,
# Transform text into list of strigs

corpora = file_test['text_'].astype(str).values.tolist()
# Use the words obtained in training to encode in testing
X = vectorizer.transform(corpora)
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
rating = pd.get_dummies(file_test['rating'])
category = pd.get_dummies(file_test['category'])
df = pd.DataFrame(category)
df = df.join(rating)
X = X.join(df)

In [114]:
y_hat = lr.predict(X)

In [115]:
df = pd.DataFrame(y_hat)
df = df.reset_index()
df.rename(columns={0: 'real review?', 'index':'ID'}, inplace=True)
print(df)
pd.DataFrame(df).to_csv("Desktop/fake-reviews/submission.csv", index=False) # IS THIS RIGHT

        ID  real review?
0        0           1.0
1        1           1.0
2        2           0.0
3        3           1.0
4        4           1.0
...    ...           ...
2244  2244           1.0
2245  2245           1.0
2246  2246           0.0
2247  2247           0.0
2248  2248           0.0

[2249 rows x 2 columns]
